In [18]:
# from helper import TimeSeriesDataset
from model import TCNClassifier
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import wandb
from sklearn.metrics import balanced_accuracy_score
import os
import random
import pandas as pd
import copy
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from helper import prepare_data

In [19]:
# # Replace YOUR_API_KEY with your actual API key
# os.environ['WANDB_API_KEY'] = 'b62c5332280f3f3dec6354dacedf36d490db3aeb'
# wandb.login()

In [20]:
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

dir_path = '/Users/charlesmiller/Documents/Code/DL_experiments/project_c/models/bfc_TCN/data'


device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [21]:
def prepare_model(params,features_cont,features_cat,save_model = False,model_name = 'tcn_rain'):
    # Hyper-parameters:
    tcl_num = params['tcl_num']
    tcl_channel_size = params['tcl_channel_size']
    # temporal casual layer channels
    channel_sizes = [tcl_channel_size] * tcl_num
    # convolution kernel size
    kernel_size = params['kernel_size']
    dropout = params['dropout']
    slices = 1
    use_bias = True
    lr = params['learning_rate']

    model_params = {
        'num_inputs':   len(features_cont) + len(features_cat),
        'num_classes':  2,
        'num_channels': channel_sizes,
        'kernel_size':  kernel_size,
        'dropout':      dropout,
        'slices':       slices,
        'act':          'relu',
        'use_bias':     use_bias
    }
    model = TCNClassifier(**model_params)
    return model

In [22]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    # wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after {str(example_ct).zfill(5)} examples: {loss:.3f}")

In [23]:
def train_model(model, X_train, y_train, X_test, y_test, criterion, optimizer, config, save_model = True, model_name = 'best_model'):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    # wandb.watch(model, criterion, log="all", log_freq=10)
    model = model.to(device)
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)

    training_loss = []
    validation_loss = []
    example_ct = 0
    min_val_loss = 100000
    batch_size = 128
    epochs = config['epochs']

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    # train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    # val_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
    
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            # Move tensors to the right device
            inputs, labels = inputs.to(device), labels.to(device)

            prediction = model(inputs)
            loss = criterion(prediction, labels)
            example_ct +=  len(labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            val_prediction = model(inputs)
            val_loss = criterion(val_prediction, labels)

            training_loss.append(loss.item())
            validation_loss.append(val_loss.item())

        if val_loss.item() < min_val_loss:
            best_params = copy.deepcopy(model.state_dict())
            min_val_loss = val_loss.item()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Train loss: {loss.item()}, Validation loss: {val_loss.item()}")
            # train_log(loss, example_ct, epoch)

    if save_model:
        torch.save(best_params, f'{dir_path}/{model_name}.pth')

    return min_val_loss

def build_optimizer(model, config):
    if config['optimizer'] == 'adam':
        optimizer = torch.optim.Adam(
            model.parameters(), lr=config['learning_rate'])
    elif config['optimizer'] == 'sgd':
        optimizer = torch.optim.SGD(
            model.parameters(), lr=config['learning_rate'])
    return optimizer

def make(config,features_cont,features_cat):
    # df = pd.read_csv(data_path)
    # alerts_df = pd.read_csv(alerts_path)
    # df = df.sort_values('t',ascending=True).reset_index(drop=True)
    # train_idx = int(len(df) * 0.8)
    # train = df.iloc[:train_idx]
    # test = df.iloc[train_idx:]
    # X_train, y_train = prepare_data(train,features_cont,features_cat,window_size=config['window_size'],alerts_df=alerts_df)
    # X_test, y_test = prepare_data(test,features_cont,features_cat,window_size=config['window_size'],alerts_df=alerts_df)
    model = prepare_model(config, features_cont, features_cat)
    criterion = nn.CrossEntropyLoss()
    optimizer = build_optimizer(model, config)
    
    return model, criterion, optimizer


In [24]:
# def test(config):
#     location = 'Sydney'
#     w = 14
#     # We pick 14 before the 2016-01-01 for the sliding window
#     from_date = '2015-12-17'
#     to_date = '2017-01-01'

#     date_fmt = '%Y-%m-%d'
#     df = get_df_complete()

#     # features
#     features_cont = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am',
#                     'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am',
#                     'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']
#     features_cat = ['RainToday']

#     # Assuming df is your DataFrame


#     X, Y = [], []
#     df_l = df[(df['Location'] == location) & (df.index < to_date) & (df.index > from_date)]
#     D = []
#     for f in features_cont:
#         D.append(df_l[f].interpolate('linear').fillna(0).values)
#     for f in features_cat:
#         D.append(df_l[f].map({'Yes': 1, 'No': 0}).fillna(0).values)
#         # transpose to time series
#     TS = []
#     for i in range(df_l.shape[0]):
#         row = []
#         for c in D:
#             row.append(c[i])
#         TS.append(row)
#     in_seq, out_seq = sliding_window(TS, w, 1)
#     rain_seq = [r[0][-1] for r in out_seq]
#     X.extend(in_seq)
#     Y.extend(rain_seq)

#     # X[features_cont] = scaler.fit_transform(X[features_cont])
#     X_test = torch.tensor(X).float().transpose(1, 2)

#     model_name = 'best_model'
#     model = prepare_model(config, features_cont, features_cat, model_name = model_name)
#     model = model.to(device)
#     model.load_state_dict(torch.load(f'{dir_path}/{model_name}.pth'))
#     model.eval()
#     X_test = X_test.to(device)

#     with torch.no_grad():
#         predicted_prob = model(X_test)
#         model_prediction = predicted_prob.data.max(1, keepdim = True)[1].view(-1).tolist()
#         no_rain_prediction = [0] * len(Y)
#         no_sun_prediction = [1] * len(Y)
#         coin_flip_prediction = [random.randint(0, 1) for _ in range(len(Y))]
#         tomorrow_like_today_prediction = X_test[:, -1, -1].view(-1).tolist()

#         ba_sc = balanced_accuracy_score
#         model_score = round(ba_sc(Y, model_prediction), 4)
#         no_rain_score = round(ba_sc(Y, no_rain_prediction), 4)
#         no_sun_score = round(ba_sc(Y, no_sun_prediction), 4)
#         coin_flip_score = round(ba_sc(Y, coin_flip_prediction), 4)
#         tlt_score = round(ba_sc(Y, tomorrow_like_today_prediction), 4)

#         print(f'Model Prediction Score: {model_score}')
#         print(f'No Rain Prediction Score: {no_rain_score}')
#         print(f'Rain Prediction Score: {no_sun_score}')
#         print(f'Coin Flip Prediction Score: {coin_flip_score}')
#         print(f'Tomorrow like Today Prediction Score: {tlt_score}')

#     torch.onnx.export(model, X_test, "model.onnx")
#     wandb.save("model.onnx")

In [29]:
def model_pipeline(hyperparameters,features_cont,features_cat,X_train, y_train, X_test, y_test):

    # # tell wandb to get started
    # with wandb.init(project="bfc-tcn", config=hyperparameters):
    #   # access all HPs through wandb.config, so logging matches execution!
    #   config = wandb.config

    # make the model, data, and optimization problem
    model,criterion, optimizer = make(hyperparameters,features_cont,features_cat)
    print(model)

    # and use them to train the model
    train_model(model, X_train, y_train, X_test, y_test, criterion, optimizer, hyperparameters)

    # # and test its final performance
    # test(config)

    return model

In [37]:
config = {
    "tcl_num":          2,
    "tcl_channel_size": 128,
    "kernel_size":      7,
    "dropout":          0.3,
    "slices":           1,
    "use_bias":         True,
    "learning_rate":    0.01,
    "epochs": 800,
    "classes":2,
    "batch_size": 128,
    "window_size": 20,
    "dataset" : "BFC3D_CLASSIFIER",
    "architecture": "TCN",
    "optimizer": "adam"
  }
features_cont = ['v', 'vw', 'o', 'c', 'h', 'l','n','sma_20','sma_5', 'bb_spread',
       'bb_trend','sma_20_trend', 'sma_5_trend', 'pct_5d_high','rsi','macd','roc',
       'pct_5d_low', 'stddev_close_diff_5d', 'stddev_close_diff_10d']
features_cat = ['bb_category']

In [28]:
data_path='/Users/charlesmiller/Documents/ts_data/day_aggs/sample_30K.csv'
alerts_path='/Users/charlesmiller/Documents/model_tester_data/BF/2015-01-01_2023-12-23BF3.csv'
df = pd.read_csv(data_path)
alerts_df = pd.read_csv(alerts_path)
df = df.sort_values('t',ascending=True).reset_index(drop=True)
train_idx = int(len(df) * 0.8)
train = df.iloc[:train_idx]
test = df.iloc[train_idx:]
X_train, y_train = prepare_data(train,features_cont,features_cat,window_size=config['window_size'],alerts_df=alerts_df)
X_test, y_test = prepare_data(test,features_cont,features_cat,window_size=config['window_size'],alerts_df=alerts_df)

Skipping PYPL-2015-07-21-12.0 due to insufficient data
Skipping PYPL-2015-08-12-12.0 due to insufficient data
Skipping PYPL-2015-08-13-12.0 due to insufficient data
Skipping PYPL-2015-07-31-15.0 due to insufficient data
Skipping PYPL-2015-07-31-13.0 due to insufficient data
Skipping PYPL-2015-08-12-13.0 due to insufficient data
Skipping PYPL-2015-08-03-13.0 due to insufficient data
Skipping PYPL-2015-08-12-11.0 due to insufficient data
Skipping SQ-2015-12-09-11.0 due to insufficient data
Skipping SQ-2015-12-14-11.0 due to insufficient data
Skipping SQ-2015-12-08-14.0 due to insufficient data
Skipping SQ-2015-11-23-13.0 due to insufficient data
Skipping SQ-2015-11-30-14.0 due to insufficient data
Skipping SQ-2015-12-02-12.0 due to insufficient data
Skipping SQ-2015-12-08-13.0 due to insufficient data
Skipping SQ-2015-11-30-13.0 due to insufficient data
Skipping SQ-2015-12-11-10.0 due to insufficient data
Skipping DOCU-2018-05-04-15.0 due to insufficient data
Skipping DOCU-2018-05-04-14.

In [38]:
model = model_pipeline(
    config,features_cont,features_cat,X_train, y_train, X_test, y_test)

TCNClassifier(
  (tcn): TemporalConvolutionNetwork(
    (network): Sequential(
      (0): TemporalCasualLayer(
        (net): Sequential(
          (conv1): Conv1d(21, 128, kernel_size=(7,), stride=(1,), padding=(6,))
          (crop1): Crop()
          (act1): ReLU()
          (dropout1): Dropout(p=0.3, inplace=False)
        )
        (bias): Conv1d(21, 128, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalCasualLayer(
        (net): Sequential(
          (conv1): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(12,), dilation=(2,))
          (crop1): Crop()
          (act1): ReLU()
          (dropout1): Dropout(p=0.3, inplace=False)
        )
        (relu): ReLU()
      )
    )
  )
  (linear): Linear(in_features=128, out_features=2, bias=True)
)
Epoch 0: Train loss: 0.665570080280304, Validation loss: 0.6084348559379578
Epoch 10: Train loss: 0.6817315816879272, Validation loss: 0.5785516500473022
Epoch 20: Train loss: 0.6805120706558228, Vali